In [3]:
import geopandas as gpd
import osmnx as ox
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import contextily as ctx
from sklearn.neighbors import KernelDensity


In [2]:
# Step 1: Get Berlin Boundary
berlin_boundary = ox.geocode_to_gdf("Berlin, Germany")

In [3]:
gdf = gpd.read_file('purchases.geojson')  # Replace with your GeoJSON file path

In [4]:
gdf = gdf[gdf['BruttoSumme'].notnull() & (gdf['BruttoSumme'] > 0)]

In [5]:
print(f"Number of points after filtering/clipping: {len(gdf)}")


Number of points after filtering/clipping: 943145


In [6]:
# Step 2: Reproject to Web Mercator
gdf = gdf.to_crs("EPSG:3857")  # Web Mercator projection for visualization

In [7]:
# Step 3: Extract coordinates and weights for KDE
coords = np.vstack([gdf.geometry.x, gdf.geometry.y])
weights = gdf['BruttoSumme']  # Replace with the correct column name

In [8]:
# Ensure no NaN or inf values
valid_mask = np.isfinite(coords).all(axis=0) & np.isfinite(weights)
coords = coords[:, valid_mask]
weights = weights[valid_mask]

In [9]:
# Step 5: Perform KDE
kde = gaussian_kde(coords, weights = weights)
xmin, ymin, xmax, ymax = berlin_boundary.total_bounds
X, Y = np.meshgrid(np.linspace(xmin, xmax, 200), np.linspace(ymin, ymax, 200))
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(kde(positions).T, X.shape)

In [ ]:
# Step 5: Plot KDE on the map
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(Z, extent=[xmin, xmax, ymin, ymax], origin='lower', cmap='Reds', alpha=0.6)
gdf.plot(ax=ax, markersize=1, color="blue", alpha=0.5)  # Plot original points
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)  # Add basemap
ax.set_title("Purchase Value Density (Valid Values Only)")
plt.colorbar(ax.imshow(Z, extent=[xmin, xmax, ymin, ymax], origin='lower', cmap='Reds', alpha=0.6), label='Density')
plt.show()

In [4]:
import geopandas as gpd
import numpy as np
from sklearn.neighbors import KernelDensity
import folium
from folium.plugins import HeatMap
from shapely.geometry import box